<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 3.6: Generators: Types | 生成器：类型
**Prev:上一小节： [Object Oriented Programming | 面向对象编程](3.5_object_oriented_programming.ipynb)**<br>
**Next:下一小节： [Introduction to FIRRTL | 介绍 FIRRTL](4.1_firrtl_ast.ipynb)**

## Motivation | 动机
Scala is a strongly-typed programming language.   
This is a two-edged sword; on one hand, many programs that would compile and execute in Python (a dynamically-typed language) would fail at compile time in Scala.
On the other hand, programs that compile in Scala will contain many fewer runtime errors than a similar Python program. 

In this section, our goal is to familiarize you with types as a first class citizen in Scala. 
While initially you may feel you have limited productivity, you will soon learn to understand compile-time error messages and how to architect your programs with the type system in mind to catch more errors for you. 

Scala 是一个强类型的编程语言。这是一把双刃剑：一方面，许多程序如果用 Python 写就可以正常编译和执行，但是用 Scala 在编译的时候就会失败。在另外一方面，使用 Scala 编译过的程序在运行的时候会产生比 Python 编译过的程序错误更少。

在本小节中，我们的目标就是让你熟悉作为 Scala 语言中的第一等公民——类型。有可能在一开始会觉得你的生产力被限制了，但是很快你就会学会理解编译时候产生的错误信息，并且知道在构建你的程序的时候怎么样借用类型系统抓住更多的编程错误。


## Setup | 设置

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

---
# Static Types | 静态类型<a name="types-in-scala"></a>

## Types in Scala | Scala 中的类型

All objects in Scala have a type, which is usually the object's class. 
Let's see some: 

所有在 Scala 中的对象都有一个类型，这通常是对象的类。让我们来看一些例子：

In [ ]:
println(10.getClass)
println(10.0.getClass)
println("ten".getClass)

When you declare your own class, it has an associated type. 

当你声明你自己的类时，它有一个关联的类型。

In [ ]:
class MyClass {
    def myMethod = ???
}
println(new MyClass().getClass)

While not required, it is HIGHLY recommended that you **define input and output types for all function declarations**. 
This will let the Scala compiler catch improper use of a function. 

尽管没有要求，但是强烈建议你**为所有的函数声明定义输入和输出类型**。这会让 Scala 编译器捕捉到不正确的函数调用。

In [ ]:
def double(s: String): String = s + s
// Uncomment the code below to test it
// double("hi")      // Proper use of double 
                     // 正确的调用方法
// double(10)        // Bad input argument! 
                     // 错误的参数输入
// double("hi") / 10 // Inproper use of double's output! 
                     //错误的使用 double 函数的输出！

Functions that don't return anything return type `Unit`. 

不返回任何值的函数将会返回 `Unit` 类型

In [ ]:
var counter = 0
def increment(): Unit = {
    counter += 1
}
increment()

## Scala vs. Chisel Types | Scala 类型与 Chisel 类型对比<a name="scala-vs-chisel-types"></a>

Recap: Module 2.2 discussed the difference between Chisel types and Scala types, for example the fact that 

简要重述：在 2.2 小节我们讨论了 Chisel 类型和 Scala 类型的区别，比如说：
```scala
val a = Wire(UInt(4.W))
a := 0.U
```
is legal because `0.U` is of type `UInt` (a Chisel type), 

whereas 是合法的，因为 `0.U` 的类型是 `UInt` (一个 Chisel 类型)
```scala
val a = Wire(UInt(4.W))
a := 0
```
is illegal because 0 is type `Int` (a Scala type). 

是非法的因为 0 的类型是 `Int` (一个 Scala 类型)。

This is also true of `Bool`, a Chisel type which is distinct from `Boolean`. 

同样 Chisel 类型 `Bool` 也是如此，区别于 Scala 类型 `Boolean`。
```scala
val bool = Wire(Bool())
val boolean: Boolean = false
// legal 合法
when (bool) { ... }
if (boolean) { ... }
// illegal 非法
if (bool) { ... }
when (boolean) { ... }
```

If you make a mistake and mix up `UInt` and `Int` or `Bool` and `Boolean`, the Scala compiler will generally catch it for you.
This is because of Scala's static typing. 
At compile time, the compiler is able to distinguish between Chisel and Scala types and understand that `if ()` expects a `Boolean` and `when ()` expects a `Bool`. 

如果你错误地将 `UInt` 与 `Int` 或者 `Bool` 与 `Boolean` 搞混淆的话， Scala 编译器将会知道并且告诉你这个错误。因为 Scala 是静态输出的。
在编译时，编译器能够区别 Chisel 和 Scala 类型，并且理解 `if ()` 接收 `Boolean`，`when ()` 接收 `Bool`。


## Scala Type Coercion | Scala 类型强调<a name="type-coercion"></a>

<!-- typeOf. Scala has a function called `typeOf[T]` which returns a type object for `T`.     typeOf: Scala 有一个函数叫做 `typeOf[T]`，这将会返回 `T` 的类型对象。 -->
<!-- This doesn't actually seem useful to Chisel users... 但是这实际上不一定对 Chisel 用户有帮助。-->

### asInstanceOf

`x.asInstanceOf[T]` casts the object `x` to the type `T`. It throws an exception if the given object cannot be cast to type `T`. 

`x.asInstanceOf[T]` 可以将对象 `x` 转换为类型 `T`。如果不能转变到类型 `T`，那么将会报错。 

In [ ]:
val x: UInt = 3.U
try {
  println(x.asInstanceOf[Int])
} catch {
  case e: java.lang.ClassCastException => println("As expected, we can't cast UInt to Int 正如预期的，我们不能够把 UInt 转换为 Int 类型。")
}

// But we can cast UInt to Data since UInt inherits from Data. 
// 但是我们可以把 UInt 转换为 Data 类型，因为 UInt 继承自 Data。
println(x.asInstanceOf[Data])


### Type Casting in Chisel | 在 Chisel 中的类型转变

The code below will give an error if you try to run it without removing the comment. 
What's the problem? 
It is trying to assign a `UInt` to an `SInt`, which is illegal. 

Chisel has a set of type casting functions.
The most general is `asTypeOf()`, which is shown below. 
Some chisel objects also define `asUInt()` and `asSInt()` as well as some others. 

If you remove the `//` from the code block below, the example should work for you. 



如果你不去注释就运行下面的代码，那么它将会报错。
问题在哪？
这是因为这段代码在尝试将一个 `UInt` 赋值给一个 `SInt`，而这是非法的。

Chisel 有一套类型转换的函数。
其中最通用的是 `asTypeOf()`，在下面你可以看到它。
一些 Chisel 对象也定义了其他的一些类型转换函数，比如 `asUInt()` 和 `asSInt()`。

如果你去掉下面这段代码中的注释，这段代码就可以正常运行了。

In [ ]:
class TypeConvertDemo extends Module {
    val io = IO(new Bundle {
        val in  = Input(UInt(4.W))
        val out = Output(SInt(4.W))
    })
    io.out := io.in//.asTypeOf(io.out)
}

Driver(() => new TypeConvertDemo) { c =>
  new PeekPokeTester(c) {
      poke(c.io.in, 3)
      expect(c.io.out, 3)
      poke(c.io.in, 15)
      expect(c.io.out, -1)
  }}

---
# Type Matching | 类型匹配<a name="type-matching"></a>

## Match Operator | 匹配操作
Recall that in 3.1 the match operator was introduced. 
Type matching is especially useful when trying to write type-generic generators. 
The following example shows an example of a "generator" that can add two literals of type `UInt` or `SInt`. 
Later sections will talk more about writing type-generic generators.

**Note: there are much better and safer ways to write type-generic generators in Scala**. 

回忆我们在 3.1 小节中介绍的匹配操作。
当我们尝试编写类型通用的生成器是，类型匹配就极其有帮助了。
下面的例子展示了能够添加两个输入类型为 `UInt` 或者 `SInt` 类型的 "generator"。

**注意：在 Scala 中有更加安全有效的途径编写通用类型的生成器**。

In [ ]:
class ConstantSum(in1: Data, in2: Data) extends Module {
    val io = IO(new Bundle {
        val out = Output(in1.cloneType)
    })
    (in1, in2) match {
        case (x: UInt, y: UInt) => io.out := x + y
        case (x: SInt, y: SInt) => io.out := x + y
        case _ => throw new Exception("I give up! 我放弃！")
    }
}
println(getVerilog(dut = new ConstantSum(3.U, 4.U)))
println(getVerilog(dut = new ConstantSum(-3.S, 4.S)))
println(getVerilog(dut = new ConstantSum(3.U, 4.S)))


It is good to remember that Chisel types generally should not be value matched. 
Scala's match executes during circuit elaboration, but what you probably want is an post-elaboration comparison.   
The following gives a syntax error: 

如果你记得 Chisel 类型通常不应该是值匹配的话，那将很棒。
Scala 的匹配将会在电路生成的时候执行，但是你想要的可能是一个在执行前就有的匹配。
下面的代码将会报语法错误：

In [ ]:
class InputIsZero extends Module {
    val io = IO(new Bundle {
        val in  = Input(UInt(16.W))
        val out = Output(Bool())
    })
    io.out := (io.in match {
        // note that case 0.U is an error 
        // 注意 case 0.U 是一个错误
        case (0.U) => true.B
        case _   => false.B
    })
}
println(getVerilog(new InputIsZero))

## Unapply | 提取器
What's actually going on when you do a match? 
Why can you do fancy value matching with case classes like this: 

当你在执行匹配的时候，究竟发生了什么呢？
为什么你可以使用案例类完成一些如下的精妙的值匹配？
```scala
case class Something(a: String, b: Int)
val a = Something("A", 3)
a match {
    case Something("A", value) => value
    case Something(str, 3)     => 0
}
```

As it turns out, the companion object that is created for every case class also contains an **unapply** method, in addition to an **apply** method. 
What is an **unapply** method? 

Scala unapply methods are another form of syntactic sugar that give match statements the ability to both match on types and **extract values** from those types during the matching. 

Let's look at the following example. 
For some reason, let's say that if the generator is being pipelined, the delay is `3*totalWidth`, otherwise the delay is `2*someOtherWidth`. 
Because case classes have **unapply** defined, we can match values inside the case class, like so: 

每一个用于构建案例类的伴生对象都包含有一个 **提取器**方法和一个**构造器**方法。
什么是一个**提取器**方法呢？

提取器方法是另外一种语法糖，这能够让匹配断言在匹配类型的同时从这些类型中**提取值**。

让我们看一些其他的例子。
因为一些原因，让我们假设如果生成器是流水线的，那么延迟是 `3*totalWidth`，不然延迟就是 `2*someOtherWidth`。 
因为案例类定义了 **提取器**，我们可以像这样在案例类内部匹配值：

In [ ]:
case class SomeGeneratorParameters(
    someWidth: Int,
    someOtherWidth: Int = 10,
    pipelineMe: Boolean = false
) {
    require(someWidth >= 0)
    require(someOtherWidth >= 0)
    val totalWidth = someWidth + someOtherWidth
}

def delay(p: SomeGeneratorParameters): Int = p match {
    case sg @ SomeGeneratorParameters(_, _, true) => sg.totalWidth * 3
    case SomeGeneratorParameters(_, sw, false) => sw * 2
}

println(delay(SomeGeneratorParameters(10, 10)))
println(delay(SomeGeneratorParameters(10, 10, true)))

If you look at the `delay` function, you should note that addition to matching on the type of each character, we are also: 
- Directly reference internal values of the parameters 
- Sometimes, are matching directly on the internal values of the parameters 

如果你观察 `delay` 函数，你应该会注意到每一个字符额外的类型匹配，我们也有：
- 直接引用变量的内部值
- 有时候，直接匹配变量的内部值

These are possible due to the compiler implementing an `unapply` method. Note that unapplying the case is just syntactic sugar; e.g. the following two cases examples are equivalent: 

由于编译器实现了一个 `unapply` 方法，这些都是可能的。注意提取 case 仅仅是语法糖；比如说，下面两个 case 语句就是相同的：
```scala
case p: SomeGeneratorParameters => p.sw * 2
case SomeGeneratorParameters(_, sw, _) => sw * 2
```

In addition, there are more syntaxes and styles of matching. The following two cases are also equivalent, but the second allows you to match on internal values while still reference the parent value: 

另外，我们有更多的匹配语法和风格。下面两种匹配的方法也是相同的，但是第二种允许你在引用父对象值的同时匹配内部值。
```scala
case SomeGeneratorParameters(_, sw, true) => sw
case sg@SomeGeneratorParameters(_, sw, true) => sw
```

Finally, you can directly embed condition checking into match statements, as demonstrated by the third of these equivalent examples: 

最后，你能够直接在匹配断言中嵌入条件检查
```scala
case SomeGeneratorParameters(_, sw, false) => sw * 2
case s@SomeGeneratorParameters(_, sw, false) => s.sw * 2
case s: SomeGeneratorParameters if s.pipelineMe => s.sw * 2
```

All these syntaxes are enabled by a Scala unapply method contained in a class's companion object. If you want to unapply a class but do not want to make it a case class, you can manually implement the unapply method. The following example demonstrates how one can manually implement a class's apply and unapply methods: 

所有的这些语法都得益于一个类的伴生对象包含的提取器方法。如果你像提取一个类但是不想让它成为一个案例类的话，你可以手动地实现这个提取器方法。下面这个例子就展示了怎样手动地实现一个类的构造器和提取器方法。

In [ ]:
class Boat(val name: String, val length: Int)
object Boat {
    def unapply(b: Boat): Option[(String, Int)] = Some((b.name, b.length))
    def apply(name: String, length: Int): Boat = new Boat(name, length)
}

def getSmallBoats(seq: Seq[Boat]): Seq[Boat] = seq.filter { b =>
    b match {
        case Boat(_, length) if length < 60 => true
        case Boat(_, _) => false
    }
}

val boats = Seq(Boat("Santa Maria", 62), Boat("Pinta", 56), Boat("Nina", 50))
println(getSmallBoats(boats).map(_.name).mkString(" and ") + " are small boats!")

## Partial Functions | 局部函数
This is a brief overview; [this guide](https://twitter.github.io/scala_school/pattern-matching-and-functional-composition.html#PartialFunction) has a more detailed overview. 

Partial functions are functions that are only defined on a subset of their inputs. 
Like an option, a partial function may not have a value for a particular input. 
This can be tested with `isDefinedAt(...)`. 

Partial functions can be chained together with `orElse`. 

Note that calling a `PartialFunction` with a undefined input will result in a runtime error. This can happen, for example, if the input to the `PartialFunction` is user-defined. To be more type-safe, we recommend writing functions that return an `Option` instead. 

这是一个简介，[这个教程](https://twitter.github.io/scala_school/pattern-matching-and-functional-composition.html#PartialFunction)有更详细的讲述。

局部函数是只定义在输入的子集的函数。
就像一个选项，对于某个特定的输入，局部函数可能没有值。
这可以用 `isDefinedAt(...)` 来测试。

局部函数可以通过 `orElse` 串联起来。

注意，对一个`局部函数`输入未定义的输入值，会导致在程序运行时产生错误。比如说，如果输入到`局部函数`中的值是用户定义的，那么这种情况可能会发生。为了类型更加安全，我们建议在写函数的时候返回`Option`。

In [ ]:
// Helper function to make this cell a bit less tedious. 函数帮助这个小程序不那么冗杂。
def printAndAssert(cmd: String, result: Boolean, expected: Boolean): Unit = {
  println(s"$cmd = $result")
  assert(result == expected)
}

// Defined for -1, 2, 5, etc.
val partialFunc1: PartialFunction[Int, String] = {
  case i if (i + 1) % 3 == 0 => "Something"
}
printAndAssert("partialFunc1.isDefinedAt(2)", partialFunc1.isDefinedAt(2), true)
printAndAssert("partialFunc1.isDefinedAt(5)", partialFunc1.isDefinedAt(5), true)
printAndAssert("partialFunc1.isDefinedAt(1)", partialFunc1.isDefinedAt(1), false)
printAndAssert("partialFunc1.isDefinedAt(0)", partialFunc1.isDefinedAt(0), false)
println(s"partialFunc1(2) = ${partialFunc1(2)}")
try {
  println(partialFunc1(0))
} catch {
  case e: scala.MatchError => println("partialFunc1(0) = can't apply PartialFunctions where they are not defined")
}

// Defined for 1, 4, 7, etc.
val partialFunc2: PartialFunction[Int, String] = {
  case i if (i + 2) % 3 == 0 => "Something else"
}
printAndAssert("partialFunc2.isDefinedAt(1)", partialFunc2.isDefinedAt(1), true)
printAndAssert("partialFunc2.isDefinedAt(0)", partialFunc2.isDefinedAt(0), false)
println(s"partialFunc2(1) = ${partialFunc2(1)}")
try {
  println(partialFunc2(0))
} catch {
  case e: scala.MatchError => println("partialFunc2(0) = can't apply PartialFunctions where they are not defined")
}

val partialFunc3 = partialFunc1 orElse partialFunc2
printAndAssert("partialFunc3.isDefinedAt(0)", partialFunc3.isDefinedAt(0), false)
printAndAssert("partialFunc3.isDefinedAt(1)", partialFunc3.isDefinedAt(1), true)
printAndAssert("partialFunc3.isDefinedAt(2)", partialFunc3.isDefinedAt(2), true)
printAndAssert("partialFunc3.isDefinedAt(3)", partialFunc3.isDefinedAt(3), false)
println(s"partialFunc3(1) = ${partialFunc3(1)}")
println(s"partialFunc3(2) = ${partialFunc3(2)}")

---
# Type Safe Connections | 类型安全连接<a name="type-safe-connections"></a>

Chisel can check the type for many connections, including:   

Chisel 可以检查连接信号两端的类型，包括：
* Bool/UInt to Clock

For other types, Chisel will let you connect them, but may truncate/pad bits as appropriate. 

对于其他类型， Chisel 会允许你连接，但是可能会适当地截短、填补信号位宽。
* Bool/UInt to Bool/UInt
* Bundle to Bundle

In [ ]:
class Bundle1 extends Bundle {
  val a = UInt(8.W)
  override def cloneType = (new Bundle1).asInstanceOf[this.type]
}

class Bundle2 extends Bundle1 {
  val b = UInt(16.W)
  override def cloneType = (new Bundle2).asInstanceOf[this.type]
}

class BadTypeModule extends Module {
  val io = IO(new Bundle {
    val c  = Input(Clock())
    val in = Input(UInt(2.W))
    val out = Output(Bool())

    val bundleIn = Input(new Bundle2)
    val bundleOut = Output(new Bundle1)
  })
  
  //io.out := io.c // won't work due to different types 
  // 因为类型不同不能正常发挥作用

  // Okay, but Chisel will truncate the input width to 1 to match the output. 
  //这可以，但是 Chisel 会把输入的位宽截短到 1 以匹配输出。
  io.out := io.in

  // Compiles; Chisel will connect the common subelements of the two Bundles (in this case, 'a'). 
  //Chisel 会允许连接两个 Bundle 的子元素。
  io.bundleOut := io.bundleIn
}

println(getVerilog(new BadTypeModule))

---
# Type Generics | 泛型类型<a name="type-generics"></a>
Scala's generic types (also known as polymorphism) is very complicated, especially when coupling it with inheritance.   

This section will just get your toes wet; to understand more, check out [this tutorial](https://twitter.github.io/scala_school/type-basics.html). 

Classes can be polymorphic in their types. One good example are sequences, which require knowing what the type of the elements it contains. 

Scala 的泛型类型或者也称为多态性，是非常复杂的，特别是当把它与继承混杂在一起的时候。

这一小节只是让你简单感受一下；如果需要了解的更多，请查看[这个教程](https://twitter.github.io/scala_school/zh_cn/type-basics.html)。

类的类型可以是多态的。一个例子是序列，它需要知道内部元素的类型。

In [ ]:
val seq1 = Seq("1", "2", "3") // 类型是 Seq[String]
val seq2 = Seq(1, 2, 3)       // 类型是 Seq[Int]
val seq3 = Seq(1, "2", true)  // 类型是 Seq[Any]

Sometimes, the Scala compiler needs help determining a polymorphic type, which requires the user to explicitly put the type: 

有时，Scala 编译器需要帮助决定多态的类型，这就是需要用户显示地给出类型。

In [ ]:
//val default = Seq() // 错误！
val default = Seq[String]() // User must tell compiler that default is of type Seq[String] 
                            // 用户必须告诉编译器 default 的类型是 Seq[String]
Seq(1, "2", true).foldLeft(default){ (strings, next) =>
    next match {
        case s: String => strings ++ Seq(s)
        case _ => strings
    }
}

Functions can also be polymorphic in their input or output types. The following example defines a function that times how long it takes to run a block of code. It is parameterized based on the return type of the block of code. *Note that the `=> T` syntax encodes an anonymous function that does not have an argument list, e.g. `{ ... }` versus `{ x => ... }`.* 

函数的输入或者输出类型也可以是多态的。下面的这个例子定义了一个函数，这个函数定义了它运行一个代码块的时间。基于代码块返回值的类型，这个函数是参数化的。*注意 `=> T` 这个语法解码一个没有参数列表的匿名函数，比如 `{ ... }` versus `{ x => ... }`。*

In [ ]:
def time[T](block: => T): T = {
    val t0 = System.nanoTime()
    val result = block
    val t1 = System.nanoTime()
    val timeMillis = (t1 - t0) / 1000000.0
    println(s"Block took $timeMillis milliseconds!")
    result
}

// Adds 1 through a million
val int = time { (1 to 1000000).reduce(_ + _) }
println(s"Add 1 through a million is $int")

// Finds the largest number under a million that, in hex, contains "beef"
val string = time {
    (1 to 1000000).map(_.toHexString).filter(_.contains("beef")).last
}
println(s"The largest number under a million that has beef: $string")

## Chisel Type Hierarchy | Chisel类型继承
To write type generic code with Chisel, it is helpful to know a bit about the type hierarchy of Chisel. 

`chisel3.Data` is the base class for Chisel hardware types.  
`UInt`, `SInt`, `Vec`, `Bundle`, etc. are all instances of `Data`.  
`Data` can be used in IOs and support `:=`, wires, regs, etc.   

Registers are a good example of polymorphic code in Chisel.
Look at the implementation of `RegEnable` (a register with a `Bool` enable signal) [here](https://github.com/freechipsproject/chisel3/blob/v3.0.0/src/main/scala/chisel3/util/Reg.scala#L10). 
The apply function is templated for `[T <: Data]`, which means `RegEnable` will work for all Chisel hardware types. 

Some operations are only defined on subtypes of `Bits`, for example `+`. 
This is why you can add `UInt`s or `SInt`s but not `Bundle`s or `Vec`s. 

为了在 Chisel 中写出泛型类型的代码，我们需要了解一些 Chisel 中的类型继承的知识。

`chisel3.Data` 是 Chisel 中最基础的硬件类型。
 `UInt`, `SInt`, `Vec`, `Bundle`等都是 `Data` 类型的实例。
 `Data` 能够在 IO 中使用并且支持 `:=` 连线、寄存器等。
 
寄存器是 Chisel 多态化的好例子。
看看 `RegEnable` 的实现 (一个带有 `Bool` 使能信号的寄存器) [这里](https://github.com/freechipsproject/chisel3/blob/v3.0.0/src/main/scala/chisel3/util/Reg.scala#L10)
应用函数被模板化为 `[T <: Data]`，这意味着 `RegEnable` 可以为任意的 Chisel 硬件类型所用。

一些操作只定义在特定的子类型`Bits`，比如说 `+`。
这就是为什么你可以把 `UInt` 或者 `SInt` 相加 但不可以让 `Bundle` 或者 `Vec` 相加。

<span style="color:blue">**Example: Type Generic ShiftRegister | 举例：泛型类型 移位寄存器**<a name="type-generic-shift-register"></a></span><br>
In Scala, objects and functions aren't the only things we can treat as parameters. 
We can also treat types as parameters. 

We usually need to provide a type constraint. 
In this case, we want to be able to put objects in a bundle, connect (:=) them, and create registers with them (RegNext). 
These operations cannot be done on arbitrary objects; for example wire := 3 is illegal because 3 is a Scala Int, not a Chisel UInt. 
If we use a type constraint to say that type T is a subclass of Data, then we can use := on any objects of type T because := is defined for all Data. 

Here is an implementations of a shift register that take types as a parameter. 
*gen* is an argument of type T that tells what width to use, for example new ShiftRegister(UInt(4.W)) is a shift register for 4-bit UInts. 
*gen* also allows the Scala compiler to infer the type T- you can write new ShiftRegister[UInt](UInt(4.W)) if you want to to be more specific, but the Scala compiler is smart enough to figure it out if you leave out the [UInt]. 

在 Scala 中，不仅仅可以把对象和函数当作参数。
我们也可以把类型当作参数。

我们经常需要提供类型约束。
在这样的类型中，我们想要把对象放进 bundle，使用 `:=` 连接它们，并且使用 `RegNext` 寄存起来。
这些操作不能被任意对象完成；比如 `wire := 3` 是非法的，因为 `3` 是 Scala Int 类型，不是 Chisel UInt 类型。
如果我们使用类型约束告诉编译器 `type T` 是 `Data` 的子类，那么我们可以在任意 `type T` 对象中使用 `:=`，因为 `:=` 被所有 `Data` 类型定义。

下面这个移位寄存器的实现就是接受类型作为参数。
*gen* 是一个类型 T 的参数，它指明了使用的位宽，例如 `new ShiftRegister(UInt(4.W))` 是一个 4 比特位宽的无符号整数移位寄存器。
*gen* 同时也允许 Scala 编译器推断类型 T。如果你想更加定义明确一些的话你也可以写 `new ShiftRegister[UInt](UInt(4.W))`。但是如果你不写 [UInt]也是完全可以的，因为 Scala 编译器足够聪明以推断出这个类型。

In [ ]:
class ShiftRegisterIO[T <: Data](gen: T, n: Int) extends Bundle {
    require (n >= 0, "Shift register must have non-negative shift")
    
    val in = Input(gen.cloneType)
    val out = Output(Vec(n + 1, gen.cloneType)) // + 1 because in is included in out 多一个因为包括输出
    override def cloneType: this.type = (new ShiftRegisterIO(gen, n)).asInstanceOf[this.type]
}

class ShiftRegister[T <: Data](gen: T, n: Int) extends Module {
    val io = IO(new ShiftRegisterIO(gen, n))
    
    io.out.foldLeft(io.in) { case (in, out) =>
        out := in
        RegNext(in)
    }
}

class ShiftRegisterTester[T <: Bits](c: ShiftRegister[T]) extends PeekPokeTester(c) {
    println(s"Testing ShiftRegister of type ${c.io.in} and depth ${c.io.out.length}")
    for (i <- 0 until 10) {
        poke(c.io.in, i)
        println(s"$i: ${peek(c.io.out)}")
        step(1)
    }
}

Driver(() => new ShiftRegister(UInt(4.W), 5)) { c => new ShiftRegisterTester(c) }
Driver(() => new ShiftRegister(SInt(6.W), 3)) { c => new ShiftRegisterTester(c) }

We generally recommend avoiding to use inheritance with type generics. 
It can be very tricky to do properly and can get frustrating quickly. 

我们建议你避免使用带有泛型类型的继承。
因为这很难做到准确所以可能很快就放弃了。

## Type Generics with Typeclasses | 带有案例类的泛型类型

The example above was limited to simple operations that could be performed on any instance of `Data` such as `:=` or `RegNext()`. 
When generating DSP circuits, we would like to do mathematical operations like addition and multiplication. 
The `dsptools` library provides tools for writing type parameterized DSP generators. 

Here is an example of writing a multiply-accumulate module. 
It can be used to generate a multiply-accumulate (MAC) for `FixedPoint`, `SInt`, or even `DspComplex[T]` (the complex number type provided by `dsptools`). 
The syntax of the type bound is a little different because `dsptools` uses typeclasses. 
They are beyond the scope of this notebook. 
Read the `dsptools` readme and documentation for more information on using typeclasses. 

`T <: Data : Ring` means that `T` is a subtype of `Data` and is also a `Ring` . 
`Ring` is defined in `dsptools` as a number with `+` and `*` (among other operations). 

_An alternative to `Ring` would be `Real`, but that would not allow us to make a MAC for `DspComplex()` because complex numbers are not `Real`._ 

上述的例子被局限在简单的操作中，这些操作是能够用于任何实例化的`Data`，例如 `:=` 或者 `RegNext()`。
当生成 DSP 电路的时候，我们想做一下数学操作比如加或者乘。
`dsptools`库提供了一些工具以使得编写类型作为变量的 DSP 生成器。

这里有一些编写乘-累加单元的例子。
它可以被用作`FixedPoint`， `SInt`， 甚至 `DspComplex[T]` (`dsptools`提供的一种复杂的数据类型) 类型的乘-累加(MAC)生成器。
类型边界的语法可能有一些不一样，因为 `dsptools` 使用的是案例类。
这些内容超过了本教程的界限。
请阅读 `dsptools` 的 readme 和其他文档以学习使用案例类。

`T <: Data : Ring` 意味着 `T` 是 `Data` 的一个子类型，同时也是 `Ring` 类型。
`Ring` 在 `dsptools` 中被定义为带有 `+` 以及 `*` (以及其他操作符) 的数字。

*另外一个代替 `Ring` 类型的是 `Real`，但是这不允许我们编写一个可以接受 `DspComplex()` 类型的乘-累加器，因为 `DspComplex()` 不是 `Real` 类型。*

In [ ]:
import chisel3.experimental._
import dsptools.numbers._

class Mac[T <: Data : Ring](genIn : T, genOut: T) extends Module {
    val io = IO(new Bundle {
        val a = Input(genIn.cloneType)
        val b = Input(genIn.cloneType)
        val c = Input(genIn.cloneType)
        val out = Output(genOut.cloneType)
    })
    io.out := io.a * io.b + io.c
}

println(getVerilog(new Mac(UInt(4.W), UInt(6.W)) ))
println(getVerilog(new Mac(SInt(4.W), SInt(6.W)) ))
println(getVerilog(new Mac(FixedPoint(4.W, 3.BP), FixedPoint(6.W, 4.BP))))


<span style="color:red">**Exercise: Mac as Object | 乘-累加对象**</span><br>

The Mac `Module` has a small number of inputs and just one output. 
It might be convenient for other Chisel generators to write code like 

乘-累加 `Module` 只有很少的输入和一个输出。
对于其他 Chisel 生成器来说像下面这样写代码就非常方便了：
```scala
val out = Mac(a, b, c)
```

Implement an `apply` method in the `Mac` companion object below that implements the `Mac` functionality. 

在下方的 `Mac` 伴生对象中实现 `apple` 对象，实现 `Mac` 的功能。


In [ ]:
object Mac {
    def apply[T <: Data : Ring](a: T, b: T, c: T): T = {
    }
}

class MacTestModule extends Module {
    val io = IO(new Bundle {
        val uin = Input(UInt(4.W))
        val uout = Output(UInt())
        val sin = Input(SInt(4.W))
        val sout = Output(SInt())
        //val fin = Input(FixedPoint(16.W, 12.BP))
        //val fout = Output(FixedPoint())
    })
    // for each IO pair, do out = in * in + in
    io.uout := Mac(io.uin, io.uin, io.uin)
    io.sout := Mac(io.sin, io.sin, io.sin)
    //io.fout := Mac(io.fin, io.fin, io.fin)
}
println(getVerilog(new MacTestModule))

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-1" />
<label for="check-1"><strong>Solution | 解答</strong> (click to toggle displaying)</label>
<article>
<pre style="background-color:#f7f7f7">
object Mac {
    def apply\[T <: Data : Ring\](a: T, b: T, c: T): T = {
        // you can also instantiate the Mac from above and connect the IOs to arguments
        a * b + c
    }
}

</pre></article></div></section></div>

<span style="color:red">**Exercise: Integrator | 练习：积分**</span><br>
Implement an integrator as pictured below. $n_1$ is the width of `genReg` and $n_2$ is the width of `genIn`.  

Don't forget that `Reg`, `RegInit`, `RegNext`, `RegEnable`, etc. are templated for types `T <: Data`. 

实现如下图所示的积分器。$n_1$ 是 `genReg` 的宽度， $n_2$ 是 `genIn` 的宽度。

不要忘记 `Reg`， `RegInit`， `RegNext`， `RegEnable` 是类型 `T <: Data` 的子类型。

<img src="images/integrator.svg" alt="Integrator" style="width: 250px;"/>

In [ ]:
class Integrator[T <: Data : Ring](genIn: T, genReg: T) extends Module {
    val io = IO(new Bundle {
        val in  = Input(genIn.cloneType)
        val out = Output(genReg.cloneType)
    })
    
    val reg = RegInit(genReg, Ring[T].zero) // init to zero
    reg := reg + io.in
    io.out := reg
}

class IntegratorSIntTester(c: Integrator[SInt]) extends PeekPokeTester(c) {
    poke(c.io.in, 3)
    expect(c.io.out, 0)
    step(1)
    poke(c.io.in, -4)
    expect(c.io.out, 3)
    step(1)
    poke(c.io.in, 6)
    expect(c.io.out, -1)
    step(1)
    expect(c.io.out, 5)
}

chisel3.iotesters.Driver(() => new Integrator(SInt(4.W), SInt(8.W))) { c => new IntegratorSIntTester(c) }

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-2" />
<label for="check-2"><strong>Solution | 解答</strong> (click to toggle displaying)</label>
<article>
<pre style="background-color:#f7f7f7">

class Integrator\[T <: Data : Ring\](genIn: T, genReg: T) extends Module {
    val io = IO(new Bundle {
        val in  = Input(genIn.cloneType)
        val out = Output(genReg.cloneType)
    })
    
    val reg = RegInit(genReg, Ring[T].zero) // init to zero
    reg := reg + io.in
    io.out := reg
}

</pre></article></div></section></div>

---
# Creating a Custom Type | 创建一个自定义类型<a name="creating-a-custom-type"></a>

One of the things that makes Chisel powerful is its extensibility. 
You can add your own types that have their own operations and representations that are tailored to your application. 
This section will introduce ways to make custom types. 

使得 Chisel 很有效率的一个特性是可扩展性。
你能够添加你自己的类型以及运算，并且定制到你的应用中去。
本小节会介绍一些自定义类型的方法。

<span style="color:blue">**Example: DspComplex 举例：DspComplex**</span><br>
`DspComplex` is a custom data type defined in **dsptools** [here](https://github.com/ucb-bar/dsptools/blob/v1.0.0/src/main/scala/dsptools/numbers/chisel_concrete/DspComplex.scala#L59). 
The key line to understand is this: 

`DspComplex` 是一个在 **dsptools** 中自定义的类型。[这里](https://github.com/ucb-bar/dsptools/blob/v1.0.0/src/main/scala/dsptools/numbers/chisel_concrete/DspComplex.scala#L59)。
其中最关键的一行代码是：
```scala
class DspComplex[T <: Data:Ring](val real: T, val imag: T) extends Bundle { ... }
```

`DspComplex` is a type-generic container. 
That means the real and imaginary parts of a complex number can be any type as long as they satisfy the type constraints, given by `T <: Data : Ring`.

`T <: Data` means `T` is a subtype of `chisel3.Data`, the base type for Chisel objects.
This means that `DspComplex` only works for objects that are Chisel types and not arbitrary Scala types. 

`T : Ring` means that a Ring typeclass implementation for `T` exists. 
`Ring` typeclasses define `+` and `*` operators as well as additive and multiplicative identities (see [this Wikipedia article](https://en.wikipedia.org/wiki/Ring_(mathematics)) for details about rings).
**dsptools** defines typeclasses for commonly used Chisel types [here](https://github.com/ucb-bar/dsptools/tree/v1.0.0/src/main/scala/dsptools/numbers/chisel_types). 
**dsptools** defines typeclasses for commonly used Chisel types [here](https://github.com/ucb-bar/dsptools/tree/v1.0.0/src/main/scala/dsptools/numbers/chisel_types) 

**dsptools** also defines a `Ring` typeclass for `DspComplex`, so we can reuse our MAC generator with complex numbers: 

`DspComplex` 是一个泛性类型容器。
这意味着只要复数的实部和虚部在类型限制内是任意类型的。

`T <: Data` 意味着 `T` 是 `chisel3.Data` 的子类型，也即是 Chisel 对象的基本类型而不是任意的 Scala 类型。
这说明 `DspComplex` 只在 Chisel 对象中有效，
 
`T : Ring` 说明存在一个 `T`类型的 `Ring` 类型类。
`Ring` 类型类定义了 `+` 和 `*` 操作符包括可加性、可乘性等。 (参加 [这篇维基文章](https://zh.wikipedia.org/wiki/%E7%8E%AF_(%E4%BB%A3%E6%95%B0)) 阅读关于“环”的详细内容)。
**dsptools** 定义了通常可以使用的 Chisel 类型 [这里](https://github.com/ucb-bar/dsptools/tree/master/src/main/scala/dsptools/numbers/chisel_types)

**dsptools** 也定义了一个 `DspComplex`类型的 `Ring` 类型类，所以我们可以将复数输入到乘-累加器中去。

In [ ]:
println(getVerilog(new Mac(DspComplex(SInt(4.W), SInt(4.W)), DspComplex(SInt(6.W), SInt(6.W))) ))

<span style="color:red">**Exercise: Sign-magnitude Numbers | 练习：原码数**</span><br>
Suppose you wanted to use a sign-magnitude representation and want to reuse all of your DSP generators. 
Typeclasses enable this kind of ad-hoc polymorphism. 
The following example gives the beggining of an implementation of a SignMagnitude type as well as an implementation of a `Ring` typeclass that will allow the type to be used with the Mac generator. 

Fill in implementations for `+` and `*`. 
You should pattern them after the implementation for `unary_-()`. 
The next block contains a test that checks the correctness of a `Mac` that uses `SignMagnitude`. 

假设你想要使用一个原码表示并且想复用所有你的 DSP 生成器。
类型类使得这样的特定多态成为可能。
下面的例子给出了实现原码类型的开始，以及一个 `Ring` 的类型类以支持被乘-累加生成器使用。

在实现中添加 `+` 和 `*`
你应该模仿 `unary_-()` 的实现方式。
下一个代码块包含一个检查使用 `SignMagnitude` 的 `Mac` 准确性。

In [ ]:
class SignMagnitude(val magnitudeWidth: Option[Int] = None) extends Bundle {
    val sign = Bool()
    val magnitude = magnitudeWidth match {
        case Some(w) => UInt(w.W)
        case None    => UInt()
    }
    def +(that: SignMagnitude): SignMagnitude = {
        // Implement this!
    }
    def -(that: SignMagnitude): SignMagnitude = {
        this.+(-that)
    }
    def unary_-(): SignMagnitude = {
        val result = Wire(new SignMagnitude())
        result.sign := !this.sign
        result.magnitude := this.magnitude
        result
    }
    def *(that: SignMagnitude): SignMagnitude = {
        // Implement this! 实现它！
    }
    override def cloneType: this.type = new SignMagnitude(magnitudeWidth).asInstanceOf[this.type]
}
trait SignMagnitudeRing extends Ring[SignMagnitude] {
    def plus(f: SignMagnitude, g: SignMagnitude): SignMagnitude = {
        f + g
    }
    def times(f: SignMagnitude, g: SignMagnitude): SignMagnitude = {
        f * g
    }
    def one: SignMagnitude = {
        val one = Wire(new SignMagnitude(Some(1)))
        one.sign := false.B
        one.magnitude := 1.U
        one
    }
    def zero: SignMagnitude = {
        val zero = Wire(new SignMagnitude(Some(0)))
        zero.sign := false.B
        zero.magnitude := 0.U
        zero
    }
    def negate(f: SignMagnitude): SignMagnitude = {
        -f
    }
    
    // Leave unimplemented for this example 
    // 让这个例子未实现的
    def minusContext(f: SignMagnitude, g: SignMagnitude): SignMagnitude = ???
    def negateContext(f: SignMagnitude): SignMagnitude = ???
    def plusContext(f: SignMagnitude,g: SignMagnitude): SignMagnitude = ???
    def timesContext(f: SignMagnitude,g: SignMagnitude): SignMagnitude = ???
}
implicit object SignMagnitudeRingImpl extends SignMagnitudeRing

In [ ]:
class SignMagnitudeMACTester(c: Mac[SignMagnitude]) extends PeekPokeTester(c) {
    // 3 * 3 + 2 = 11
    poke(c.io.a.sign, 0)
    poke(c.io.a.magnitude, 3)
    poke(c.io.b.sign, 0)
    poke(c.io.b.magnitude, 3)
    poke(c.io.c.sign, 0)
    poke(c.io.c.magnitude, 2)
    expect(c.io.out.sign, 0)
    expect(c.io.out.magnitude, 11)
    // 3 * 3 - 2 = 7
    poke(c.io.c.sign, 1)
    expect(c.io.out.sign, 0)
    expect(c.io.out.magnitude, 7)
    // 3 * (-3) - 2 = -11
    poke(c.io.b.sign, 1)
    expect(c.io.out.sign, 1)
    expect(c.io.out.magnitude, 11)
}
val works = iotesters.Driver(() => new Mac(new SignMagnitude(Some(4)), new SignMagnitude(Some(5)))) {
  c => new SignMagnitudeMACTester(c)
}
assert(works) // Scala Code: if works == false, will throw an error   
              // Scala 代码：如果 works == false，将会报错
println("成功！！") // Scala Code: if we get here, our tests passed!    
                    // Scala 代码：如果我们运行到了这里，那么我们的验证就通过啦！

Look at the verilog to see if the output looks reasonable: 看看生成的 Verilog 是否合理：

In [ ]:
println(getVerilog(new Mac(new SignMagnitude(Some(4)), new SignMagnitude(Some(5)))))

`SignMagnitude` even works with `DspComplex`!

In [ ]:
println(getVerilog(new Mac(DspComplex(new SignMagnitude(Some(4)), new SignMagnitude(Some(4))), DspComplex(new SignMagnitude(Some(5)), new SignMagnitude(Some(5))))))

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-3" />
<label for="check-3"><strong>Solution | 解答</strong> (click to toggle displaying)</label>
<article>
<pre style="background-color:#f7f7f7">
    // implementations for class SignMagnitude

    def +(that: SignMagnitude): SignMagnitude = {
      val result = Wire(new SignMagnitude())
      val signsTheSame = this.sign === that.sign
      when (signsTheSame) {
        result.sign      := this.sign
        result.magnitude := this.magnitude + that.magnitude
      } .otherwise {
        when (this.magnitude > that.magnitude) {
          result.sign      := this.sign
          result.magnitude := this.magnitude - that.magnitude
        } .otherwise {
          result.sign      := that.sign
          result.magnitude := that.magnitude - this.magnitude
        }   
      }   
      result
    }
    def \*(that: SignMagnitude): SignMagnitude = {
        val result = Wire(new SignMagnitude())
        result.sign := this.sign ^ that.sign
        result.magnitude := this.magnitude \* that.magnitude
        result
    }


</pre></article></div></section></div>